In [2]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [3]:
csv_file = "Resources/Public_School_Locations_201718.csv"
schools_df = pd.read_csv(csv_file)
schools_df.head()

,NCESSCH,NAME,STREET,CITY,STATE,ZIP,NMCNTY,LAT,LON,SCHOOLYEAR
0,1.023700e+10,Pillans Middle School,2051 Military Rd,Mobile,AL,36605,Mobile County,30.628471,-88.085440,2017-2018
1,1.023700e+10,Phillips Preparatory Middle School,3255 Old Shell Rd,Mobile,AL,36607,Mobile County,30.696872,-88.120320,2017-2018
2,1.023700e+10,WD Robbins Elementary School,2416 W Main St,Prichard,AL,36610,Mobile County,30.744798,-88.098192,2017-2018
3,1.023700e+10,CL Scarborough Model Middle School,1800 Phillips Ln,Mobile,AL,36618,Mobile County,30.726541,-88.143270,2017-2018
4,1.023700e+10,Semmes Middle School,4566 Ed George Road,Semmes,AL,36575,Mobile County,30.786316,-88.290427,2017-2018


In [4]:
csv_file = "Resources/Zillow_Data_3bhk_Pricing.csv"
homes_df = pd.read_csv(csv_file, engine="python")
homes_df.head()

,RegionID,ZIP_Code,City,State,Metro,CountyName,SizeRank,2017-2018
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2.064652e+06
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,5.903138e+05
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,3.819060e+06
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,2.494665e+05
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,6.947342e+05


### Create new data with select columns

In [5]:
schools_cols = ['NAME', 'STREET', 'CITY', 'STATE', 'ZIP']
schools_transformed = schools_df[schools_cols].copy()

# Rename the column headers 
schools_transformed = schools_transformed.rename(columns={"NAME": "name", 
                                                          "STREET": "street",
                                                          "CITY": "city",
                                                          "STATE": "state",
                                                          "ZIP": "zip"})
# Clean the data by dropping duplicates and setting the index
schools_transformed.drop_duplicates(inplace=True)
schools_transformed.head()

,name,street,city,state,zip
0,Pillans Middle School,2051 Military Rd,Mobile,AL,36605
1,Phillips Preparatory Middle School,3255 Old Shell Rd,Mobile,AL,36607
2,WD Robbins Elementary School,2416 W Main St,Prichard,AL,36610
3,CL Scarborough Model Middle School,1800 Phillips Ln,Mobile,AL,36618
4,Semmes Middle School,4566 Ed George Road,Semmes,AL,36575


In [6]:
homes_cols = ['ZIP_Code', 'City', 'State', 'CountyName', "2017-2018"]
homes_transformed = homes_df[homes_cols].copy()

# Rename the column headers 
homes_transformed = homes_transformed.rename(columns={"ZIP_Code": "zip", 
                                                      "City": "city",
                                                      "State": "state",
                                                      "CountyName": "county",
                                                      "2017-2018": "homeprice"})
# Clean the data by dropping duplicates and setting the index
homes_transformed.drop_duplicates(inplace=True)
homes_transformed.head()

,zip,city,state,county,homeprice
0,10025,New York,NY,New York County,2.064652e+06
1,60657,Chicago,IL,Cook County,5.903138e+05
2,10023,New York,NY,New York County,3.819060e+06
3,77494,Katy,TX,Harris County,2.494665e+05
4,60614,Chicago,IL,Cook County,6.947342e+05


### Clean DataFrame

In [7]:
# Keep data from the state of GA
schools_transformed_ga = schools_transformed[schools_transformed['state'] == 'GA']
# Sort by Zip and Reset index
schools_transformed_ga.sort_values('zip', inplace=True)
schools_transformed_ga.reset_index(drop=True, inplace=True)
schools_transformed_ga.head()

C:\Users\sagas\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,name,street,city,state,zip
0,DeKalb School of the Arts,1192 Clarendon Ave,Avondale Estates,GA,30002
1,Museum School Avondale Estates,3191 Covington Hwy,Avondale Estates,GA,30002
2,DeKalb Elementary School of the Arts,3131 Old Rockbridge Rd,Avondale Estates,GA,30002
3,Avondale Elementary School,8 Lakeshore Dr,Avondale Estates,GA,30002
4,Manning Oaks Elementary School,405 Cumming St,Alpharetta,GA,30004


In [8]:
# Keep data from the state of GA
homes_transformed_ga = homes_transformed[homes_transformed['state'] == 'GA']
# Sort by Zip and Reset index
homes_transformed_ga.sort_values('zip', inplace=True)
homes_transformed_ga.reset_index(drop=True, inplace=True)
homes_transformed_ga.head()

C:\Users\sagas\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,zip,city,state,county,homeprice
0,30002,Avondale Estates,GA,Dekalb County,362628.2500
1,30004,Milton,GA,Fulton County,277332.2500
2,30005,Alpharetta,GA,Fulton County,314515.9167
3,30008,Marietta,GA,Cobb County,163375.2500
4,30009,Alpharetta,GA,Fulton County,330000.5833


### Connect to local database

In [9]:
from config import dbuser, dbpassword
rds_connection_string = f"{dbuser}:{dbpassword}@localhost:5432/garealestate_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [10]:
engine.table_names()

['ga_schools', 'ga_homes']

### Use pandas to load csv converted DataFrame into database

In [11]:
homes_transformed_ga.to_sql(name='ga_homes', con=engine, if_exists='append', index=False)

In [12]:
schools_transformed_ga.to_sql(name='ga_schools', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the ga_schools table
* NOTE: can also check using pgAdmin

In [13]:
pd.read_sql_query('select * from ga_schools', con=engine).head()

,id,name,street,city,state,zip
0,1,DeKalb School of the Arts,1192 Clarendon Ave,Avondale Estates,GA,30002
1,2,Museum School Avondale Estates,3191 Covington Hwy,Avondale Estates,GA,30002
2,3,DeKalb Elementary School of the Arts,3131 Old Rockbridge Rd,Avondale Estates,GA,30002
3,4,Avondale Elementary School,8 Lakeshore Dr,Avondale Estates,GA,30002
4,5,Manning Oaks Elementary School,405 Cumming St,Alpharetta,GA,30004


In [14]:
pd.read_sql_query('select * from ga_homes', con=engine).head()

,zip,city,state,county,homeprice
0,30002,Avondale Estates,GA,Dekalb County,362628.2500
1,30004,Milton,GA,Fulton County,277332.2500
2,30005,Alpharetta,GA,Fulton County,314515.9167
3,30008,Marietta,GA,Cobb County,163375.2500
4,30009,Alpharetta,GA,Fulton County,330000.5833


In [17]:
pd.read_sql_query("""SELECT gs.name, gh.city, gh.county, gh.homeprice 
                  FROM ga_schools AS gs 
                  JOIN ga_homes AS gh 
                  ON gs.zip = gh.zip;""", con=engine).head()

,name,city,county,homeprice
0,DeKalb School of the Arts,Avondale Estates,Dekalb County,362628.25
1,Museum School Avondale Estates,Avondale Estates,Dekalb County,362628.25
2,DeKalb Elementary School of the Arts,Avondale Estates,Dekalb County,362628.25
3,Avondale Elementary School,Avondale Estates,Dekalb County,362628.25
4,Manning Oaks Elementary School,Milton,Fulton County,277332.25
